In [73]:
%matplotlib inline
import pandas as pd
import numpy as np

In [74]:
# column names, is_guest_login & dis_host_login & logged_in & land & flag & service & protocol_type
with open("C:/Users/Bin/Documents/Datasets/KDD99/columns.txt") as col_file:
    line = col_file.readline()
#line = line.replace('.',',')
columns = line.split('.')
col_names = []
col_types = []
for col in columns:
    col_names.append(col.split(': ')[0].strip())
    col_types.append(col.split(': ')[1])
col_names.append("label")

In [75]:
#import 10% collection samples
df = pd.read_csv("C:/Users/Bin/Documents/Datasets/KDD99/kddcup.data_10_percent_corrected",names=col_names)

In [76]:
continuous = df.iloc[:,np.array(pd.Series(col_types)=="continuous")]
label = df.iloc[:,-1]


In [77]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(continuous)
cont = scaler.transform(continuous)

In [78]:
cont = pd.DataFrame(cont)
cont.columns = continuous.columns.values


In [108]:
data = pd.concat((cont,label),axis=1)

In [109]:
# split data according to window length
L = 20
n_list = []
a_list = []
temp = []

for index, row in data.iterrows():
    if len(temp) ==L:
        for x in temp:
            if data.iloc[x,-1] == "normal.":
                n_list.append(x)
            else:
                a_list.append(x)
        temp.clear()
        temp.append(index)
        continue
    if len(temp) == 0:
        temp.append(index)
    elif row.label == data.iloc[temp[0],-1]:
        temp.append(index)
    else:
        temp.clear()
        temp.append(index)

In [121]:
#normal = data[(data.label=="smurf.") | (label=="normal.")].iloc[:,:-1]
#anomaly = data[(data.label!="smurf.") & (label!="normal.")].iloc[:,:-1]
normal = data.iloc[np.array(n_list),:]
anomaly = data.iloc[np.array(a_list),:]

In [126]:
x = int(normal.shape[0]/20)
sn = normal[:(x//2)*20]
vn1 = normal[(x//2)*20:(x//2)*20+(x//6)*20]
vn2 = normal[(x//2)*20+(x//6)*20:(x//2)*20+(x//3)*20]
tn = normal[(x//2)*20+(x//3)*20:]

In [127]:
y = int(anomaly.shape[0]/20)
va = anomaly[:(y//4)*20]
ta = anomaly[(y//4)*20:]

In [ ]:
root = "C:/Users/Bin/Documents/Datasets/KDD99/6_subsets_win/"
sn.to_csv(root+"training_normal.csv",index=None,header=None)
vn1.to_csv(root+"validation_1.csv.csv",index=None,header=None)
vn2.to_csv(root+"validation_2.csv.csv",index=None,header=None)
tn.to_csv(root+"test_normal.csv",index=None,header=None)
va.to_csv(root+"validation_anomaly.csv",index=None,header=None)
ta.to_csv(root+"test_anomaly.csv",index=None,header=None)